In [1]:
import numpy as np
import pandas as pd
import cv2
import os

class COVID_IMG(object):
    
    def __init__(self, path, label):
        self.path = path
        self.label = label
        
    def images_path(self):
        li_path = [img for img in os.listdir(self.path) if img.endswith(".jpg") or img.endswith(".png") or img.endswith(".jpeg")]
        #print(len(li_path))
        return li_path

    def img_read(self, li_path):
        img_values = []
        labels = []

        for img in li_path:
            img = cv2.imread(self.path+"/"+img)
            #print(img)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))
            #print(img.shape)
            img = img.flatten().tolist()
            img_values.append(img)
            labels.append(self.label)
        #print(len(img_values), labels)
        return labels, img_values
    
    def matrix_to_csv(self, image_array, li, labels, val):
        data = {}
        
        for i in range(len(image_array)):
            data[li[i]] = image_array[i]
        #print(data)
        print(val)

        if val == "covid":
            print(val)
            data = pd.DataFrame(data=data)
            data.T.to_csv("corona.csv")
            data = pd.read_csv("corona.csv")
            data['labels'] = labels
            data.to_csv("final_corona.csv")
            print("corona")
            os.remove("corona.csv")
            
        elif val == "normal":
            print(val)
            data = pd.DataFrame(data=data)
            data.T.to_csv("normal.csv")
            data = pd.read_csv("normal.csv")
            data['labels'] = labels
            data.to_csv("final_normal.csv")
            print("Normal")
            os.remove("normal.csv")
        else:
            pass
        
label = ["covid", "normal"]
for j in range(len(label)):
    print(label[j])
    covid = COVID_IMG(path=os.getcwd()+"/dataset/"+label[j], label=label[j])
    li_path = covid.images_path()
    labels, img_values = covid.img_read(li_path)
    covid.matrix_to_csv(img_values, li_path, labels, label[j])

covid
covid
covid
corona
normal
normal
normal
Normal


In [2]:
def merge(csv1, csv2):

    corona = pd.read_csv(csv1)
    normal = pd.read_csv(csv2)

    print(corona.shape)
    print(normal.shape)

    data = corona.append(normal)

    print(data.shape)

    data.to_csv("COVID.csv")

merge("final_corona.csv", "final_normal.csv")

(151, 150531)
(151, 150531)


C:\Users\tariv\AppData\Local\Temp\ipykernel_7280\2520253957.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = corona.append(normal)


(302, 150531)


In [3]:
import cv2
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import keras 
from keras.applications.resnet import ResNet152

In [15]:
data = pd.read_csv("COVID.csv")

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Columns: 150532 entries, Unnamed: 0.2 to labels
dtypes: int64(150530), object(2)
memory usage: 346.8+ MB


In [17]:
data.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,0,1,2,3,4,5,6,...,150519,150520,150521,150522,150523,150524,150525,150526,150527,labels
0,0,0,01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg,133,133,133,122,122,122,115,...,217,217,217,214,214,214,254,254,254,covid
1,1,1,1-s2.0-S0140673620303706-fx1_lrg.jpg,44,42,43,53,51,52,57,...,79,79,81,103,103,105,99,99,101,covid
2,2,2,1-s2.0-S0929664620300449-gr2_lrg-a.jpg,12,12,12,13,13,13,12,...,11,11,11,11,11,11,11,11,11,covid
3,3,3,1-s2.0-S0929664620300449-gr2_lrg-b.jpg,0,0,0,1,1,1,0,...,0,0,0,0,0,0,227,227,227,covid
4,4,4,1-s2.0-S0929664620300449-gr2_lrg-c.jpg,3,3,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,covid


In [18]:
data.drop(['Unnamed: 0.2', 'Unnamed: 0.1'], axis=1, inplace=True)

In [19]:
data = data.set_index('Unnamed: 0')

In [20]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,150519,150520,150521,150522,150523,150524,150525,150526,150527,labels
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg,133,133,133,122,122,122,115,115,115,110,...,217,217,217,214,214,214,254,254,254,covid
1-s2.0-S0140673620303706-fx1_lrg.jpg,44,42,43,53,51,52,57,55,56,58,...,79,79,81,103,103,105,99,99,101,covid
1-s2.0-S0929664620300449-gr2_lrg-a.jpg,12,12,12,13,13,13,12,12,12,13,...,11,11,11,11,11,11,11,11,11,covid
1-s2.0-S0929664620300449-gr2_lrg-b.jpg,0,0,0,1,1,1,0,0,0,1,...,0,0,0,0,0,0,227,227,227,covid
1-s2.0-S0929664620300449-gr2_lrg-c.jpg,3,3,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,covid


In [21]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [22]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [23]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((241, 150528), (61, 150528), (241,), (61,))

In [24]:
model = ResNet152(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

234698864/234698864 [==============================] - 62s 0us/step


In [25]:
model.summary()

Model: "resnet152"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                          

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block6_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block6_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block6_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block6_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block6_3_conv (Conv2D)   (None, 28, 28, 512)  66048       ['conv3_block6_2_relu[0][0]']    
                                                                                                  
 conv3_block6_3_bn (BatchNormal  (None, 28, 28, 512)  2048       ['conv3_block6_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block1_0_conv (Conv2D)   (None, 14, 14, 1024  525312      ['conv3_block8_out[0][0]']       
                                )                                                                 
                                                                                                  
 conv4_block1_3_conv (Conv2D)   (None, 14, 14, 1024  263168      ['conv4_block1_2_relu[0][0]']    
                                )                                                                 
                                                                                                  
 conv4_block1_0_bn (BatchNormal  (None, 14, 14, 1024  4096       ['conv4_block1_0_conv[0][0]']    
 ization)                       )                                                                 
          

...

                                                                                                  
 conv4_block28_1_conv (Conv2D)  (None, 14, 14, 256)  262400      ['conv4_block27_out[0][0]']      
                                                                                                  
 conv4_block28_1_bn (BatchNorma  (None, 14, 14, 256)  1024       ['conv4_block28_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block28_1_relu (Activati  (None, 14, 14, 256)  0          ['conv4_block28_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block28_2_conv (Conv2D)  (None, 14, 14, 256)  590080      ['conv4_block28_1_relu[0][0]']   
          

                                )                                                                 
                                                                                                  
 conv4_block31_1_conv (Conv2D)  (None, 14, 14, 256)  262400      ['conv4_block30_out[0][0]']      
                                                                                                  
 conv4_block31_1_bn (BatchNorma  (None, 14, 14, 256)  1024       ['conv4_block31_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block31_1_relu (Activati  (None, 14, 14, 256)  0          ['conv4_block31_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_blo

 conv4_block33_out (Activation)  (None, 14, 14, 1024  0          ['conv4_block33_add[0][0]']      
                                )                                                                 
                                                                                                  
 conv4_block34_1_conv (Conv2D)  (None, 14, 14, 256)  262400      ['conv4_block33_out[0][0]']      
                                                                                                  
 conv4_block34_1_bn (BatchNorma  (None, 14, 14, 256)  1024       ['conv4_block34_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block34_1_relu (Activati  (None, 14, 14, 256)  0          ['conv4_block34_1_bn[0][0]']     
 on)                                                                                              
          

                                                                                                  
 conv4_block36_out (Activation)  (None, 14, 14, 1024  0          ['conv4_block36_add[0][0]']      
                                )                                                                 
                                                                                                  
 conv5_block1_1_conv (Conv2D)   (None, 7, 7, 512)    524800      ['conv4_block36_out[0][0]']      
                                                                                                  
 conv5_block1_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block1_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block1_1_bn[0][0]']      
 n)       

                                                                  'conv5_block3_3_bn[0][0]']      
                                                                                                  
 conv5_block3_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block3_add[0][0]']       
                                                                                                  
Total params: 58,370,944
Trainable params: 58,219,520
Non-trainable params: 151,424
__________________________________________________________________________________________________


In [26]:
from keras.models import Model

In [28]:
# model=Model(inputs=model.input,outputs=preds)

import tensorflow as tf

from tensorflow.keras import layers, models

In [29]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
Total params: 56,320
Trainable params: 56,320
Non-traina

In [31]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0

In [35]:
model.compile(optimizer='adam',
              loss='Categorical_crossentropy',
              metrics=['accuracy'])


In [36]:
X_train = X_train.reshape(len(X_train),224,224,3)

In [37]:
X_test = X_test.reshape(len(X_test),224,224,3)

In [38]:
X_train.shape

(241, 224, 224, 3)

In [39]:
type(X_train)

numpy.ndarray

In [40]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

In [41]:
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

In [43]:
history = model.fit( x=X_train, y=y_train, steps_per_epoch=200,epochs=10,validation_data=(X_test, y_test),validation_steps=10)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\tariv\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\tariv\anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\tariv\anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\tariv\anaconda3\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\tariv\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\tariv\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 